In [3]:
import requests
import pandas as pd
from newspaper import Article

# 🔍 네이버 뉴스 검색
def search_naver_news_all(query, target_dates):
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    all_filtered = []
    for start in range(1, 500, 100):
        params = {
            "query": query,
            "display": 100,
            "start": start,
            "sort": "sim"
        }
        try:
            response = requests.get("https://openapi.naver.com/v1/search/news.json", headers=headers, params=params)
            response.raise_for_status()
        except:
            break
        items = response.json().get("items", [])
        for item in items:
            try:
                pubdate = parsedate_to_datetime(item["pubDate"]).date()
                if pubdate in target_dates:
                    all_filtered.append(item["originallink"].replace("amp;", ""))
            except:
                continue
    return all_filtered

# 📄 기사 본문 추출
def extract_article_text(url):
    try:
        article = Article(url, language="ko")
        article.download()
        article.parse()
        return article.text
    except:
        return None

In [6]:
text = extract_article_text('https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90+%EC%82%AC%EA%B8%B0&sort=0&ds=2024.01.01&de=2025.06.01&nso=so%3Ar%2Cp%3Afrom20240101to20250601%2Ca%3A&start=1%20%EC%97%AC%EA%B8%B0%EC%84%9C%20%EB%89%B4%EC%8A%A4%20%EA%B0%80%EC%A0%B8%EC%99%80%EC%84%9C%20%EC%A0%95%EB%A6%AC%ED%95%B4%EC%A4%98')
print(text)

“이런 사기 꾼들에게 경제를 맡기겠느냐”고 했다. 그는 파업 노동자에 대한 기업의 손해배상 청구를 제한하는 노란봉투법(노동조합 및 노동관계조정법 개정안)을 예로 들며 “법을 어겨서 파업하는데 손해배상을 안 하면 이 나라가 무법천지가 되는 거 아니겠느냐”고 했다. 김 후보는 평택 삼성전자 반도체...


In [3]:
import requests

def fetch_news_from_webhook(query: str, date: str, news_office_checked: str) -> dict:
    """n8n을 통해 supabase에 뉴스를 저장하는 함수
    
    Args:
        query (str): 검색어
        date (str): 날짜 (YYYY.MM.DD 형식)
        news_office_checked (str): 뉴스사 ID
          1023(조선일보)
          1025(중앙일보)
          1020(동아일보)
          1015(한국경제)
          1009(매일경제)
          1011(서울경제)
        
    Returns:
        dict: 응답 데이터
          sucess : 성공했거나 이미 중복 데이터거나
          retry : 실패 > 재시도
    """
    url = "https://moluvalu.app.n8n.cloud/webhook/3fc6b155-45d8-42cb-b54b-c81bd87ac445"
    params = {
        "query": query,
        "date": date,
        "news_office_checked": news_office_checked
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"요청 중 오류가 발생했습니다: {e}")
        return {}


In [9]:
from datetime import datetime, timedelta
import time

# 시작 날짜 설정 (2025.06.30)
current_date = datetime(2025, 4, 27)

# 검색어와 뉴스사 리스트 설정
query = "사기"
news_offices = ["1015", "1009", "1011"] # 한국경제, 매일경제, 서울경제

# 과거로 이동하면서 데이터 수집
while current_date.year >= 2025:  # 2025년까지 수집
    
    # 날짜 형식 변환 (YYYY.MM.DD)
    date_str = current_date.strftime("%Y.%m.%d")
    
    # 각 뉴스사별로 데이터 수집
    for news_office in news_offices:
        print(f"수집 중: {date_str} - 뉴스사 {news_office}")
        
        # API 호출 및 재시도 로직
        max_retries = 3
        retry_count = 0
        
        while retry_count < max_retries:
            result = fetch_news_from_webhook(
                query=query,
                date=date_str,
                news_office_checked=news_office
            )
            
            # retry가 있으면 재시도
            if result.get('retry'):
                print(f"재시도 {retry_count + 1}/{max_retries}")
                retry_count += 1
                time.sleep(2)  # 재시도 전 2초 대기
                continue
            else:
                break
                
        # 결과 출력
        print(f"{date_str} {news_office} 결과: {result}")
        
        # API 호출 간 간격 두기 
        time.sleep(2)
    
    # 하루 전으로 이동
    current_date -= timedelta(days=1)
    
print("데이터 수집 완료")
# 현재 날짜와 시간을 포함한 로그 파일명 생성
log_filename = datetime.now().strftime("%Y%m%d_%H%M%S_log.txt")




수집 중: 2025.06.30 - 뉴스사 1015
2025.06.30 1015 결과: {'output': '2025.06.30 에는 데이터가 없습니다'}
수집 중: 2025.06.30 - 뉴스사 1009
2025.06.30 1009 결과: {'output': 'success'}
수집 중: 2025.06.30 - 뉴스사 1011
2025.06.30 1011 결과: {'output': 'success'}
수집 중: 2025.06.29 - 뉴스사 1015
2025.06.29 1015 결과: {'output': '네, 이해했습니다. 다음과 같이 작업을 처리하겠습니다.\n\n1. 입력받은 JSON이 없으면 "2025.06.29 에는 데이터가 없습니다"를 반환합니다.\n2. 성공적으로 데이터 입력 시 success, 실패 시 retry를 반환합니다.\n3. 이미 동일한 source(중복 URL)가 있으면 success만 반환합니다.\n4. 각 컬럼 매핑도 명확히 적용합니다.\n5. 미래에셋 주식코드는 매핑하지 않으며, 필터 지침도 반영하겠습니다.\n\n이제 입력할 JSON 데이터를 제공해주시면 저장 절차를 진행하겠습니다!'}
수집 중: 2025.06.29 - 뉴스사 1009
2025.06.29 1009 결과: {'output': 'sucess\n\n주어진 롯데케미칼 뉴스 데이터를 정상적으로 Supabase에 입력했습니다.'}
수집 중: 2025.06.29 - 뉴스사 1011
2025.06.29 1011 결과: {'output': '2025.06.29 에는 데이터가 없습니다'}
수집 중: 2025.06.28 - 뉴스사 1015
2025.06.28 1015 결과: {'output': '"2025.06.28 에는 데이터가 없습니다"'}
수집 중: 2025.06.28 - 뉴스사 1009
2025.06.28 1009 결과: {'output': '2025.06.28 에는 데이터가 없습니다'}
수집 중: 2025.06.28 - 뉴스사 1011
2025.06.28 1011 결과: {'

KeyboardInterrupt: 